<a href="https://colab.research.google.com/github/Sudarshan-1/RandomPhoto/blob/main/Movie(recom).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
import numpy as np
import pandas as pd

df = pd.read_csv("movies.csv")
print(df)

       movieId                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
3            4            Waiting to Exhale (1995)   
4            5  Father of the Bride Part II (1995)   
...        ...                                 ...   
62418   209157                           We (2018)   
62419   209159           Window of the Soul (2001)   
62420   209163                    Bad Poems (2018)   
62421   209169                 A Girl Thing (2001)   
62422   209171      Women of Devil's Island (1962)   

                                            genres  
0      Adventure|Animation|Children|Comedy|Fantasy  
1                       Adventure|Children|Fantasy  
2                                   Comedy|Romance  
3                             Comedy|Drama|Romance  
4                                           Comedy  
...                              

In [128]:
import re #library for regular expression

def clean_title(title): #take movie title as input and clean it
    title = re.sub("[^a-zA-Z0-9 ]", "", title) #remove all the extra characters other than required ones
    return title

#comvert titles to different sets in 0 and 1 

In [129]:
df["clean_title"]=df["title"].apply(clean_title) #creating a new column 

In [130]:
df

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [131]:
#build using frequency matrix 
#match the column with the frequency and the movie and then to turn titles to numbers for easy reference

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer(ngram_range=(1,2)) #searches two words or numbers or words and numbers together for faster search
tfidf= vectorizer.fit_transform(df['clean_title']) #it helps to convert the titles to matrix


In [132]:
from scipy.linalg.special_matrices import dft
#search function giving the result of the search function
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
 
  title=clean_title(title)
  query_vec=vectorizer.transform([title]) #compare the query with each of the titles for comparison
  similarity=cosine_similarity(query_vec,tfidf).flatten()
  indices=np.argpartition(similarity,-5)[-5:] #it gets 5 most similar titles to the our search term to make it easy
  results=df.iloc[indices][::-1] #gives the titles that are required
  return results





In [133]:
results

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
3021,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 1999
59767,201588,Toy Story 4 (2019),Adventure|Animation|Children|Comedy,Toy Story 4 2019
14813,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,Toy Story 3 2010
20497,106022,Toy Story of Terror (2013),Animation|Children|Comedy,Toy Story of Terror 2013


In [134]:
indices #gives the most similar search terms

array([20497, 14813, 59767,  3021,     0])

In [135]:
similarity #compares harry potter with all the other titles to see for a match

array([1.        , 0.09681098, 0.06531543, ..., 0.        , 0.        ,
       0.        ])

In [136]:
query_vec

<1x170073 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [137]:
import ipywidgets as widgets
from IPython.display import display

df_input=widgets.Text(
    value="Toy Story",
    description="Movie title",
    disabled=False
)
df_list=widgets.Output()
def on_type(data):
  with df_list: #this is the output widget
    df_list.clear_output() #clear the output widget that is present 
    title=data["new"]#give the new value
    if len(title)>5:
      display(search(title)) #search and then display
df_input.observe(on_type,names="value")
display(df_input,df_list)






Text(value='Toy Story', description='Movie title')

Output()

In [138]:
ratings=pd.read_csv("ratings.csv")
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [139]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [140]:
movie_Id=2
userId=1

In [141]:
#users who liked the same movie and gave a good rating
similar_users=ratings[(ratings["movieId"]==movie_Id)&(ratings["rating"]>4)]["userId"].unique()
similar_users


array([     9,    258,    333, ..., 162424, 162440, 162498])

In [142]:
similar_user_recs=ratings[(ratings["userId"].isin(similar_users))&(ratings["rating"]>4)]  #to find the similar movies that users went to and have given ratings >4 
similar_user_recs

,userId,movieId,rating,timestamp
1459,9,2,5.0,859383142
1460,9,10,5.0,859383403
1465,9,116,5.0,859382367
1468,9,161,5.0,859382967
1470,9,168,5.0,859383520
...,...,...,...,...
24988759,162498,76251,5.0,1315030638
24988761,162498,77846,4.5,1315031262
24988767,162498,84952,4.5,1318586450
24988771,162498,88129,5.0,1338756094


In [143]:
#similar to one that have given five star tenpercent of users
similar_user_recs.value_counts()/len(similar_users) #this shows the count for the number of times it has appeared
similar_user_recs=similar_user_recs[similar_user_recs>.1] #for greater then ten percent
similar_user_recs




,userId,movieId,rating,timestamp
1459,9,2,5.0,859383142
1460,9,10,5.0,859383403
1465,9,116,5.0,859382367
1468,9,161,5.0,859382967
1470,9,168,5.0,859383520
...,...,...,...,...
24988759,162498,76251,5.0,1315030638
24988761,162498,77846,4.5,1315031262
24988767,162498,84952,4.5,1318586450
24988771,162498,88129,5.0,1338756094


In [144]:
all_users=ratings[(ratings["movieId"].isin(similar_user_recs.index)&(ratings["rating"]>4))] #anyone who has liked a movie from the list above
all_users







,userId,movieId,rating,timestamp
127,2,1580,4.5,1141417059
128,2,1584,5.0,1141416806
934,4,1580,4.5,1573938142
1127,4,152081,4.5,1573941094
1439,8,1580,5.0,890510873
...,...,...,...,...
24992661,162516,1584,4.5,1175652132
24996418,162518,1584,5.0,869634476
24997082,162521,1479,4.5,1207988767
24997881,162528,1584,5.0,929127272


In [145]:
#percentage of users recommended each of the movies
all_users_recs=all_users["movieId"].value_counts()/len(all_users['userId'].unique())
all_users_recs



1580      0.294509
1625      0.231065
1584      0.215244
152081    0.123403
1544      0.049585
            ...   
159749    0.000040
107354    0.000040
192587    0.000040
88473     0.000040
94044     0.000040
Name: movieId, Length: 234, dtype: float64

In [146]:
#creating a recommendation score
rec_percentages = pd.concat([similar_user_recs, all_users_recs], axis=1)
rec_percentages.columns = ["similar", "all","adf","dfg","drt"]
rec_percentages



,similar,all,adf,dfg,drt
1459,9,2,5.0,859383142,0.011695
1460,9,10,5.0,859383403,0.001602
1465,9,116,5.0,859382367,0.005407
1468,9,161,5.0,859382967,0.002844
1470,9,168,5.0,859383520,0.000561
...,...,...,...,...,...
24988759,162498,76251,5.0,1315030638,NaN
24988761,162498,77846,4.5,1315031262,NaN
24988767,162498,84952,4.5,1318586450,NaN
24988771,162498,88129,5.0,1338756094,NaN


In [147]:
rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]


In [148]:
rec_percentages=rec_percentages.sort_values("score", ascending=False) #how much the average user liked a movie
rec_percentages 

,similar,all,adf,dfg,drt,score
24960446,162318,1,5.0,848349759,NaN,162318.000000
24823819,161378,1,5.0,1312016674,NaN,161378.000000
24770251,160983,1,5.0,1486594832,NaN,160983.000000
24721865,160684,1,5.0,833130255,NaN,160684.000000
24721741,160682,1,5.0,1503121305,NaN,160682.000000
...,...,...,...,...,...,...
67709,541,193687,4.5,1542827247,NaN,0.002793
47935,388,160980,4.5,1504136585,0.000160,0.002410
47936,388,165551,4.5,1504136577,NaN,0.002344
47937,388,167380,5.0,1504136600,0.001121,0.002318


In [149]:
rec_percentages.head(10).merge(df, left_index=True, right_on="movieId")

,similar,all,adf,dfg,drt,score,movieId,title,genres,clean_title


In [150]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [151]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()